<a href="https://colab.research.google.com/github/sbs23003/CA-Contstruction-Industry/blob/main/datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


# Housing Prices dataset - Columns 'Quarter', 'Value' where 'Type of Employee' = "All employees" and 'Economic Sector NACE Rev 2' = "Construction (F)"
#df_housingPrices = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPA09/CSV/1.0/en')
#df_rentPrices = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/RIA02/CSV/1.0/en')
#df_GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_5454986.csv')

# Labour Costs dataset - Columns 'Quarter', 'Value' where 'Type of Employee' = "All employees" and 'Economic Sector NACE Rev 2' = "Construction (F)"
df_labourCosts = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/EHQ03/CSV/1.0/en')

# Construction Workers dataset - Columns 'Quarter', 'Value' where 'Sex' = "Both Sexes"
#df_constructionWorkers = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/QLF03/CSV/1.0/en')

# Buildings Built dataset - Columns 'Year' and 'Values'
#df_buildingsBuilt = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/NDA02/CSV/1.0/en')

#df_buildingsBuiltEircodes = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/NDA01/CSV/1.0/en')
#df_immigration = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/PEA15/CSV/1.0/en')
#df_wages = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPA09/CSV/1.0/en')
#df_population = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/PEA11/CSV/1.0/en')
#print(df_buildingsBuiltEircodes)

<ipython-input-115-493fe1b84fc6>:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labourCosts = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/EHQ03/CSV/1.0/en')


In [ ]:
new_df = pd.DataFrame()
import re

#Wages
print(df_labourCosts)
"""
dff_labourCosts = df_labourCosts[(df_labourCosts["Economic Sector NACE Rev 2"] == 'Construction (F)')
                                  & (df_labourCosts["Type of Employee"] == 'All employees')]

#print(dff_labourCosts['Quarter'].str.split('Q').str[0])
dff_labourCosts['Quarter'] = dff_labourCosts['Quarter'].str.split('Q').str[0]
dff_labourCosts['Quarter'] = dff_labourCosts['Quarter'].astype(int)

dff_labourCosts['VALUE'] = dff_labourCosts['VALUE'].astype(float)
#.str.replace('Q1', '').str.replace('Q2', '').str.replace('Q3', '').str.replace('Q4', '')
#dff_labourCosts['Statistic Label'].unique()


#filtered_labourCosts = dff_labourCosts[(dff_labourCosts["Statistic Label"] ==  'Average Hourly Earnings')]
filtered_employment = dff_labourCosts[(dff_labourCosts["Statistic Label"] ==  'Employment')]
#filtered_labourCosts
filtered_employment
#filtered_rentPrices['VALUE'] = filtered_rentPrices['VALUE'].astype(float)

#filtered_rentPrices.info()

#dff_labourCosts

#df2_mean = dff_labourCosts.groupby(['Quarter'])['VALUE'].mean().reset_index()
#df2_mean
#dff_labourCosts.head(20)

#dff_rentPrices
#print(dff_rentPrices['Location'].unique())
#updated_locations = [location.split(", ")[-1] for location in locations]

#filtered_rentPrices = df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
#                                    & (df_rentPrices["Property Type"] ==  'All property types')]

#filtered_rentPrices
#df2_mean = filtered_housingPrices.groupby('Year').mean().reset_index()
#df2_mean
#df_housingPrices
#new_df = pd.merge(df_buildingsBuilt)
"""

      STATISTIC                                    Statistic Label  TLIST(Q1)  \
0      EHQ03C01                                         Employment      20081   
1      EHQ03C01                                         Employment      20081   
2      EHQ03C01                                         Employment      20081   
3      EHQ03C01                                         Employment      20081   
4      EHQ03C01                                         Employment      20081   
...         ...                                                ...        ...   
85675   EHQ03S9  Average Hourly Benefit in Kind (Seasonally Adj...      20224   
85676   EHQ03S9  Average Hourly Benefit in Kind (Seasonally Adj...      20224   
85677   EHQ03S9  Average Hourly Benefit in Kind (Seasonally Adj...      20224   
85678   EHQ03S9  Average Hourly Benefit in Kind (Seasonally Adj...      20224   
85679   EHQ03S9  Average Hourly Benefit in Kind (Seasonally Adj...      20224   

      Quarter C02665V03225 

'\ndff_labourCosts = df_labourCosts[(df_labourCosts["Economic Sector NACE Rev 2"] == \'Construction (F)\')\n                                  & (df_labourCosts["Type of Employee"] == \'All employees\')]\n\n#print(dff_labourCosts[\'Quarter\'].str.split(\'Q\').str[0])\ndff_labourCosts[\'Quarter\'] = dff_labourCosts[\'Quarter\'].str.split(\'Q\').str[0]\ndff_labourCosts[\'Quarter\'] = dff_labourCosts[\'Quarter\'].astype(int)\n\ndff_labourCosts[\'VALUE\'] = dff_labourCosts[\'VALUE\'].astype(float)\n#.str.replace(\'Q1\', \'\').str.replace(\'Q2\', \'\').str.replace(\'Q3\', \'\').str.replace(\'Q4\', \'\')\n#dff_labourCosts[\'Statistic Label\'].unique()\n\n\n#filtered_labourCosts = dff_labourCosts[(dff_labourCosts["Statistic Label"] ==  \'Average Hourly Earnings\')]\nfiltered_employment = dff_labourCosts[(dff_labourCosts["Statistic Label"] ==  \'Employment\')]\n#filtered_labourCosts\nfiltered_employment\n#filtered_rentPrices[\'VALUE\'] = filtered_rentPrices[\'VALUE\'].astype(float)\n\n#filtered

In [ ]:
new_df = pd.DataFrame()
import re

#housing prices
"""
filtered_housingPrices = df_housingPrices[(df_housingPrices["Type of Dwelling"] == 'All Dwelling Types') 
                    & (df_housingPrices["Dwelling Status"] == 'All Dwelling Statuses') 
                    & (df_housingPrices["Sectoral Flow"] == 'All sectoral flow types') 
                    & (df_housingPrices["Type of Sale"] == 'All Sale Types')
                    & (df_housingPrices["Statistic Label"] == 'Mean Sale Price')]

df2_mean = filtered_housingPrices.groupby('Year').mean().reset_index()
print(df2_mean)
"""

#rent prices
"""
dff_rentPrices = df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
                                 & (df_rentPrices["Property Type"] ==  'All property types')]
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.replace(r'\b( Town| City)\b', '', regex=True)
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.split(', ').str[-1]
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.replace(' ', '', case=False)
dff_rentPrices['Location'] =  dff_rentPrices['Location'].str.split(',', n=1).str[0]
dff_rentPrices.loc[dff_rentPrices['Location'].str.contains('Dublin', case=False), 'Location'] = 'Dublin'
df2_mean = dff_rentPrices.groupby(['Year'])['VALUE'].mean().reset_index()
print(df2_mean)
"""

#GDP
"""
data = {
    'Year': [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
    'GDP (USD)': [253.31, 217.98, 224.24, 231.34, 244.72, 249.03, 271.92, 294.80, 326.99, 348.28, 385.58, 385.94, 360.91, 504.2, 578]
}

df_GDP = pd.DataFrame(data)
df_GDP
"""


#Population
"""
filter_population = df_population[(df_population['Year'] > 2009) 
                                  & (df_population['Single Year of Age'] == 'All ages') 
                                   & (df_population['Sex'] == 'Both sexes')]
filter_population
"""

#Wage
#Immigration
"""
filter_immigration = df_immigration[(df_immigration['Year'] > 2009) & (df_immigration['Component'] == 'Net migration')]
print(filter_immigration.info())
"""

#Number of Buildings Built
"""
filter_buildingsBuilt = df_buildingsBuilt[(df_buildingsBuilt['Year'] > 2009) 
                                  & (df_buildingsBuilt['Type of House'] == 'All house types')]
#df_buildingsBuilt
filter_buildingsBuilt

### new addition dont know if correct
#Number of Buildings Built

filter_constructionWorkers = df_constructionWorkers[(df_constructionWorkers['Sex'] == 'Both sexes') 
                                  & (df_constructionWorkers['NACE Rev 2 Economic Sector'] == 'Construction (F)')
                                  & (df_constructionWorkers['Statistic Label'] == 'Persons aged 15-89 years in Employment')]
                                  #& (df_constructionWorkers['Quarter'] > 2009 )]
#df_buildingsBuilt
#df_constructionWorkers

filter_constructionWorkers['Quarter'] = filter_constructionWorkers['Quarter'].str.split('Q').str[0].astype(int)
#filter_constructionWorkers['Quarter'] = filter_constructionWorkers['Quarter'].astype(int)

df2_mean = filter_constructionWorkers.groupby('Quarter').mean().reset_index()
df2_mean
#filter_constructionWorkers = filter_constructionWorkers[filter_constructionWorkers['Quarter'] > 2009]
#filter_constructionWorkers
"""

#Number of Construction Workers

#Labour Costs
"""
dff_labourCosts = df_labourCosts[(df_labourCosts["Economic Sector NACE Rev 2"] == 'Construction (F)')
                                  & (df_labourCosts["Type of Employee"] == 'All employees')]

#print(dff_labourCosts['Quarter'].str.split('Q').str[0])
dff_labourCosts['Quarter'] = dff_labourCosts['Quarter'].str.split('Q').str[0]
dff_labourCosts['Quarter'] = dff_labourCosts['Quarter'].astype(int)

dff_labourCosts['VALUE'] = dff_labourCosts['VALUE'].astype(float)
#.str.replace('Q1', '').str.replace('Q2', '').str.replace('Q3', '').str.replace('Q4', '')
#dff_labourCosts['Statistic Label'].unique()


#filtered_labourCosts = dff_labourCosts[(dff_labourCosts["Statistic Label"] ==  'Average Hourly Earnings')]
filtered_employment = dff_labourCosts[(dff_labourCosts["Statistic Label"] ==  'Employment')]
#filtered_labourCosts
filtered_employment
#filtered_rentPrices['VALUE'] = filtered_rentPrices['VALUE'].astype(float)

#filtered_rentPrices.info()

#dff_labourCosts
"""
#df2_mean = dff_labourCosts.groupby(['Quarter'])['VALUE'].mean().reset_index()
#df2_mean
#dff_labourCosts.head(20)

#dff_rentPrices
#print(dff_rentPrices['Location'].unique())
#updated_locations = [location.split(", ")[-1] for location in locations]

#filtered_rentPrices = df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
#                                    & (df_rentPrices["Property Type"] ==  'All property types')]

#filtered_rentPrices
#df2_mean = filtered_housingPrices.groupby('Year').mean().reset_index()
#df2_mean
#df_housingPrices
#new_df = pd.merge(df_buildingsBuilt)



In [ ]:
## Housing Prices
#TO DO - existem 1610 valores NULL na coluna VALUE. Encontrar a melhor forma de preencher este valores!! A1
# Soma dos valores null
#df_housingPrices[df_housingPrices["Type of Dwelling"].str.contains("All Dwelling Types").loc[:, ["Year", "VALUE"]]]
#df_labourCosts[df_labourCosts["Type of Employee"].str.contains("All employees")].loc[:, ["Quarter", "VALUE"]].isnull().sum()
"""
df_housingPrices[(df_housingPrices["Type of Dwelling"] == 'House') 
                    & (df_housingPrices["Dwelling Status"] == 'Existing') 
                    & (df_housingPrices["Sectoral Flow"] == 'All sectoral flow types') 
                    & (df_housingPrices["Type of Sale"] == 'All Sale Types')
                    & (df_housingPrices["Statistic Label"] == 'Mean Sale Price')]
"""
#df_housingPrices.columns
#df_housingPrices

## Labour Costs
#TO DO - existem 1610 valores NULL na coluna VALUE. Encontrar a melhor forma de preencher este valores!! A1
# Soma dos valores null
#df_labourCosts[df_labourCosts["Type of Employee"].str.contains("All employees")].loc[:, ["Quarter", "VALUE"]].isnull().sum()
#df_labourCosts


"""
locations = df_rentPrices["Location"].unique()
updated_locations = [location.split(", ")[-1] for location in locations]

# Print the updated location names
for location in updated_locations:
    print(location)
"""
df_immigration
#df_rentPrices[(df_rentPrices["Number of Bedrooms"] ==  'All bedrooms')
#              & (df_rentPrices["Property Type"] ==  'All property types')]

## Construction Workers
#TO DO - existem 307 valores NULL na coluna VALUE. Encontrar a melhor forma de preencher este valores!! A1
# Soma dos valores null
#df_constructionWorkers[df_constructionWorkers["Sex"].str.contains("Both sexes")].loc[:, ["Quarter", "VALUE"]].isnull().sum()

#df_constructionWorkers
#df_buildingsBuilt[df_buildingsBuilt["Type of House"].str.contains("All house types")].loc[:,["Year", "VALUE"]]
#df_buildingsBuilt

,STATISTIC,STATISTIC Label,TLIST(A1),Year,C02541V03076,Component,UNIT,VALUE
0,PEA15,Annual Population Change,1951,1951,1,Annual births,Thousand,NaN
1,PEA15,Annual Population Change,1951,1951,2,Annual deaths,Thousand,NaN
2,PEA15,Annual Population Change,1951,1951,3,Natural increase,Thousand,26.6
3,PEA15,Annual Population Change,1951,1951,4,Immigrants,Thousand,NaN
4,PEA15,Annual Population Change,1951,1951,5,Emigrants,Thousand,NaN
...,...,...,...,...,...,...,...,...
571,PEA15,Annual Population Change,2022,2022,4,Immigrants,Thousand,120.7
572,PEA15,Annual Population Change,2022,2022,5,Emigrants,Thousand,59.6
573,PEA15,Annual Population Change,2022,2022,6,Net migration,Thousand,61.1
574,PEA15,Annual Population Change,2022,2022,7,Population change,Thousand,88.8
